In [1]:
# Data manipulation
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from lightgbm import plot_importance

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import optuna

In [2]:
train_x = pd.DataFrame(pd.read_csv("C:/Users/yjs49/Desktop/ecar_EDA/imputation/탄소배출/탄소배출_0108.csv"))
train_y = pd.DataFrame(pd.read_csv("C:/Users/yjs49/Desktop/ecar_EDA/imputation/탄소배출/탄소배출_0912.csv"))

guro_x = pd.DataFrame(pd.read_csv("C:/Users/yjs49/Desktop/ecar_EDA/imputation/탄소배출/구로구_210108.csv"))
guro_y = pd.DataFrame(pd.read_csv("C:/Users/yjs49/Desktop/ecar_EDA/imputation/탄소배출/구로구_210912.csv"))

In [3]:
train_x

,행정동,1월가스,1월온실가스,2월가스,2월온실가스,3월가스,3월온실가스,4월가스,4월온실가스,5월가스,5월온실가스,6월가스,6월온실가스,7월가스,7월온실가스,8월가스,8월온실가스
0,소공동,3.161366e+06,6886.390826,2.606075e+06,5676.803770,1.570709e+06,3421.469238,1.117806e+06,2434.912575,1.198502e+06,2610.693396,1.586075e+06,3454.940352,1.695213e+06,3692.676731,1.999751e+06,4356.050271
1,회현동,2.368467e+06,5159.223772,1.994571e+06,4344.767323,1.183125e+06,2577.196451,7.117657e+05,1550.436611,7.393899e+05,1610.610156,8.577629e+05,1868.461731,9.352972e+05,2037.354497,1.017586e+06,2216.603303
2,명동,4.818737e+06,10496.636373,4.196608e+06,9141.454669,2.698417e+06,5877.951317,1.633553e+06,3558.363246,1.639096e+06,3570.436309,2.053234e+06,4472.550993,2.298793e+06,5007.452576,2.655031e+06,5783.443159
3,필동,9.091701e+05,1980.441908,8.868064e+05,1931.727112,6.871323e+05,1496.777649,4.551528e+05,991.457622,3.575049e+05,778.751495,3.965561e+05,863.816592,4.105893e+05,894.385235,4.217499e+05,918.696184
4,장충동,1.353651e+06,2948.652858,1.372374e+06,2989.436261,9.475743e+05,2064.097668,6.912783e+05,1505.808885,5.646304e+05,1229.932206,5.352743e+05,1165.985934,5.227659e+05,1138.738925,5.454677e+05,1188.190339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,세곡동,1.283392e+06,2795.607248,1.179694e+06,2569.721984,9.470552e+05,2062.966915,7.984701e+05,1739.304465,4.683278e+05,1020.156726,3.034470e+05,660.997499,2.718180e+05,592.100237,2.617532e+05,570.176043
1632,일원본동,1.414364e+06,3080.902875,1.428718e+06,3112.170433,1.167777e+06,2543.763778,8.833753e+05,1924.253119,6.367511e+05,1387.032480,5.962614e+05,1298.834052,6.023391e+05,1312.073085,5.635804e+05,1227.645117
1633,일원1동,5.412808e+06,11790.699264,6.088078e+06,13261.637513,3.773293e+06,8219.349795,1.165208e+06,2538.167401,2.947677e+05,642.091327,2.402098e+05,523.248097,2.073805e+05,451.736162,1.316748e+05,286.826727
1634,일원2동,1.146674e+05,249.779649,1.012180e+05,220.482873,1.167477e+05,254.311091,9.315253e+04,202.913817,9.381813e+04,204.363693,6.551830e+04,142.718267,7.867320e+04,171.373547,7.791050e+04,169.712154


In [4]:
train_y

,행정동,9월가스,9월온실가스,10월가스,10월온실가스,11월가스,11월온실가스,12월가스,12월온실가스
0,소공동,1.342386e+06,2924.115458,1.015137e+06,2211.270182,1.626306e+06,3542.576713,2.410588e+06,5250.974733
1,회현동,7.285289e+05,1586.951685,5.813149e+05,1266.276023,1.135224e+06,2472.853239,1.761175e+06,3836.361666
2,명동,1.806084e+06,3934.186642,1.417705e+06,3088.180553,2.558989e+06,5574.236451,3.811755e+06,8303.131289
3,필동,3.717927e+05,809.874709,2.811686e+05,612.468575,4.708631e+05,1025.679364,7.644311e+05,1665.157474
4,장충동,4.442521e+05,967.712611,4.446127e+05,968.498268,7.381340e+05,1607.874514,1.172913e+06,2554.952624
...,...,...,...,...,...,...,...,...,...
1631,세곡동,2.447405e+05,533.117300,3.266038e+05,711.439900,5.356916e+05,1166.895000,9.649698e+05,2101.990000
1632,일원본동,8.189000e+05,1783.807000,6.302529e+05,1372.878000,5.057316e+05,1101.633000,8.371947e+05,1823.658000
1633,일원1동,1.042601e+05,227.109500,1.618372e+05,352.529400,4.319157e+05,940.840300,5.346132e+05,1164.546000
1634,일원2동,8.617102e+04,187.706000,6.310124e+04,137.453200,8.072367e+04,175.840100,7.872372e+04,171.483600


In [5]:
guro_x

,행정동,1월가스,1월온실가스,2월가스,2월온실가스,3월가스,3월온실가스,4월가스,4월온실가스,5월가스,5월온실가스,6월가스,6월온실가스,7월가스,7월온실가스,8월가스,8월온실가스
0,신도림동,2.979134e+06,6489.435778,2.660926e+06,5796.284195,1.801233e+06,3923.618241,1.141260e+06,2486.003023,763480.306265,1663.086303,525004.229698,1143.614755,389923.689095,849.369318,311298.445476,678.100243
1,구로1동,1.381606e+06,3009.547055,1.209618e+06,2634.907409,8.678105e+05,1890.348388,5.352239e+05,1165.876165,351540.647332,765.759681,257359.373550,560.604963,192825.846868,420.031823,179970.232019,392.028485
2,구로2동,2.683260e+06,5844.935497,2.334458e+06,5085.142124,1.761741e+06,3837.593424,1.084778e+06,2362.968417,826332.628770,1799.997283,620818.482599,1352.326585,570627.255220,1242.995222,513581.129930,1118.731860
3,구로3동,2.611342e+06,5688.275725,2.162210e+06,4709.933073,1.675993e+06,3650.808896,1.188879e+06,2589.729918,939166.111369,2045.782037,758622.679814,1652.504954,741244.053364,1614.649157,745125.067285,1623.103155
4,구로4동,1.782925e+06,3883.738614,1.521106e+06,3313.418783,1.125252e+06,2451.133104,7.297497e+05,1589.610955,510736.361949,1112.535112,331206.995360,721.466963,231258.918794,503.750440,209618.823666,456.611902
5,구로5동,3.077955e+06,6704.697860,2.580908e+06,5621.982254,1.691166e+06,3683.861343,1.158290e+06,2523.099540,736413.076566,1604.125858,561356.032483,1222.799752,474897.401392,1034.467238,391710.856148,853.262297
6,가리봉동,1.093541e+06,2382.057151,9.196714e+05,2003.316748,6.868636e+05,1496.192475,4.660213e+05,1015.132534,317387.881671,691.364839,205840.512761,448.381621,142964.009281,311.417968,114366.501160,249.124123
7,고척1동,1.663826e+06,3624.306591,1.414538e+06,3081.282202,9.954745e+05,2168.438443,6.883796e+05,1499.494752,457040.350348,995.569290,351553.276102,765.787190,306243.064965,667.088126,248666.672854,541.669686
8,고척2동,1.639530e+06,3571.383024,1.370625e+06,2985.626284,9.496408e+05,2068.599079,6.198521e+05,1350.221578,415702.888631,905.524052,276462.587007,602.217422,190443.201856,414.841716,161028.464037,350.767703
9,개봉1동,2.064783e+06,4497.708228,1.775909e+06,3868.455440,1.265381e+06,2756.374414,8.279172e+05,1803.448867,534516.904872,1164.336180,342378.807425,745.802479,243259.143852,529.890486,212262.517401,462.370650


In [6]:
guro_y

,행정동,9월가스,9월온실가스,10월가스,10월온실가스,11월가스,11월온실가스,12월가스,12월온실가스
0,신도림동,331510.257541,722.127557,387783.596288,844.707561,9.565792e+05,2083.712933,1.838105e+06,4003.937457
1,구로1동,162546.178654,354.073735,299335.923434,652.042325,6.356184e+05,1384.565092,1.139868e+06,2482.970491
2,구로2동,298433.663573,650.076936,389140.709977,847.663757,7.199002e+05,1568.155823,1.147391e+06,2499.357510
3,구로3동,683740.092807,1489.388494,714546.345708,1556.493640,1.263837e+06,2753.010968,1.881811e+06,4099.141988
4,구로4동,93426.515081,203.510629,123860.895592,269.805727,2.730274e+05,594.734520,5.138114e+05,1119.233363
5,구로5동,377477.858469,822.258611,506155.125290,1102.555821,1.089248e+06,2372.704663,1.984358e+06,4322.520322
6,고척1동,274676.791183,598.327430,311343.962877,678.199393,6.901168e+05,1503.278751,1.150204e+06,2505.485497
7,고척2동,190195.299304,414.301711,368226.907193,802.107298,7.604024e+05,1656.381603,1.347584e+06,2935.437974
8,개봉1동,114863.721578,250.207216,179005.429234,389.926859,4.410182e+05,960.668375,7.575249e+05,1650.113564
9,개봉2동,211420.255220,460.535953,329455.986079,717.652746,8.596907e+05,1872.661117,1.453452e+06,3166.049389


In [7]:
### 9월까지는 데이터가 있구나!
add_y = pd.DataFrame(train_y[['9월가스','9월온실가스']])
train_x = pd.concat([train_x, add_y], axis=1)
train_y = train_y.drop(['9월가스','9월온실가스'], axis=1)

guro_add_y = pd.DataFrame(guro_y[['9월가스','9월온실가스']])
guro_x = pd.concat([guro_x, guro_add_y], axis=1)
guro_y = guro_y.drop(['9월가스','9월온실가스'], axis=1)

### 그냥 '행정동'이름도 일단은 제거!
train_x = train_x.drop(['행정동'], axis=1)
train_y = train_y.drop(['행정동'], axis=1)
guro_x = guro_x.drop(['행정동'], axis=1)
guro_y = guro_y.drop(['행정동'], axis=1)

In [8]:
train = pd.concat([train_x, train_y], axis=1)
train

,1월가스,1월온실가스,2월가스,2월온실가스,3월가스,3월온실가스,4월가스,4월온실가스,5월가스,5월온실가스,...,8월가스,8월온실가스,9월가스,9월온실가스,10월가스,10월온실가스,11월가스,11월온실가스,12월가스,12월온실가스
0,3.161366e+06,6886.390826,2.606075e+06,5676.803770,1.570709e+06,3421.469238,1.117806e+06,2434.912575,1.198502e+06,2610.693396,...,1.999751e+06,4356.050271,1.342386e+06,2924.115458,1.015137e+06,2211.270182,1.626306e+06,3542.576713,2.410588e+06,5250.974733
1,2.368467e+06,5159.223772,1.994571e+06,4344.767323,1.183125e+06,2577.196451,7.117657e+05,1550.436611,7.393899e+05,1610.610156,...,1.017586e+06,2216.603303,7.285289e+05,1586.951685,5.813149e+05,1266.276023,1.135224e+06,2472.853239,1.761175e+06,3836.361666
2,4.818737e+06,10496.636373,4.196608e+06,9141.454669,2.698417e+06,5877.951317,1.633553e+06,3558.363246,1.639096e+06,3570.436309,...,2.655031e+06,5783.443159,1.806084e+06,3934.186642,1.417705e+06,3088.180553,2.558989e+06,5574.236451,3.811755e+06,8303.131289
3,9.091701e+05,1980.441908,8.868064e+05,1931.727112,6.871323e+05,1496.777649,4.551528e+05,991.457622,3.575049e+05,778.751495,...,4.217499e+05,918.696184,3.717927e+05,809.874709,2.811686e+05,612.468575,4.708631e+05,1025.679364,7.644311e+05,1665.157474
4,1.353651e+06,2948.652858,1.372374e+06,2989.436261,9.475743e+05,2064.097668,6.912783e+05,1505.808885,5.646304e+05,1229.932206,...,5.454677e+05,1188.190339,4.442521e+05,967.712611,4.446127e+05,968.498268,7.381340e+05,1607.874514,1.172913e+06,2554.952624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,1.283392e+06,2795.607248,1.179694e+06,2569.721984,9.470552e+05,2062.966915,7.984701e+05,1739.304465,4.683278e+05,1020.156726,...,2.617532e+05,570.176043,2.447405e+05,533.117300,3.266038e+05,711.439900,5.356916e+05,1166.895000,9.649698e+05,2101.990000
1632,1.414364e+06,3080.902875,1.428718e+06,3112.170433,1.167777e+06,2543.763778,8.833753e+05,1924.253119,6.367511e+05,1387.032480,...,5.635804e+05,1227.645117,8.189000e+05,1783.807000,6.302529e+05,1372.878000,5.057316e+05,1101.633000,8.371947e+05,1823.658000
1633,5.412808e+06,11790.699264,6.088078e+06,13261.637513,3.773293e+06,8219.349795,1.165208e+06,2538.167401,2.947677e+05,642.091327,...,1.316748e+05,286.826727,1.042601e+05,227.109500,1.618372e+05,352.529400,4.319157e+05,940.840300,5.346132e+05,1164.546000
1634,1.146674e+05,249.779649,1.012180e+05,220.482873,1.167477e+05,254.311091,9.315253e+04,202.913817,9.381813e+04,204.363693,...,7.791050e+04,169.712154,8.617102e+04,187.706000,6.310124e+04,137.453200,8.072367e+04,175.840100,7.872372e+04,171.483600


In [13]:
def objective_rf(trial):
    
    params = {
        'max_depth' : trial.suggest_int('max_depth',1, 10),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 2, 1000),
        'n_estimators' : trial.suggest_int('n_estimators', 100, 500)
    }
    
    kf = KFold(n_splits = 5)
    
    score = []
    
    for train_idx, valid_idx in kf.split(train_x):
        train_data = train.iloc[train_idx]
        valid_data = train.iloc[valid_idx]
        
        
        rf = RandomForestRegressor(**params)
        model = MultiOutputRegressor(estimator=rf)
        
        x_train = train_data.iloc[:,0:18]
        y_train = train_data.iloc[:,18:24]
        model.fit(x_train,y_train)
        
        x_valid = valid_data.iloc[:,0:18]
        y_valid = valid_data.iloc[:,18:24]
        preds = model.predict(x_valid)
        
        nrmse = nrmse_1(preds, y_valid)
        score.append(nrmse)
    
    total = sum(score)/len(score)
    
    return total

In [14]:
sampler=optuna.samplers.TPESampler(seed=42)
study=optuna.create_study(
    direction='minimize',
    sampler=sampler
)
study.optimize(objective_rf, n_trials=100)
print('Best Score:',study.best_value)
print('Best trial:',study.best_trial.params)

[I 2022-10-28 15:59:39,557] A new study created in memory with name: no-name-1502191a-5ba0-4c5f-a180-4cf1d09767bc
[I 2022-10-28 16:00:29,375] Trial 0 finished with value: 1.9343047922090122 and parameters: {'max_depth': 4, 'max_leaf_nodes': 951, 'n_estimators': 393}. Best is trial 0 with value: 1.9343047922090122.
[I 2022-10-28 16:00:57,223] Trial 1 finished with value: 1.8240014186724867 and parameters: {'max_depth': 6, 'max_leaf_nodes': 157, 'n_estimators': 162}. Best is trial 1 with value: 1.8240014186724867.
[I 2022-10-28 16:01:15,007] Trial 2 finished with value: 3.2504054085606953 and parameters: {'max_depth': 1, 'max_leaf_nodes': 867, 'n_estimators': 341}. Best is trial 1 with value: 1.8240014186724867.
[I 2022-10-28 16:02:42,676] Trial 3 finished with value: 1.8562639069819178 and parameters: {'max_depth': 8, 'max_leaf_nodes': 22, 'n_estimators': 488}. Best is trial 1 with value: 1.8240014186724867.
[I 2022-10-28 16:03:30,898] Trial 4 finished with value: 1.7419096847045865 and

[I 2022-10-28 16:43:33,998] Trial 38 finished with value: 3.245838311814225 and parameters: {'max_depth': 1, 'max_leaf_nodes': 958, 'n_estimators': 366}. Best is trial 29 with value: 1.7227708338086274.
[I 2022-10-28 16:44:26,057] Trial 39 finished with value: 2.047022263982355 and parameters: {'max_depth': 3, 'max_leaf_nodes': 746, 'n_estimators': 410}. Best is trial 29 with value: 1.7227708338086274.
[I 2022-10-28 16:46:06,823] Trial 40 finished with value: 1.8071083962462677 and parameters: {'max_depth': 6, 'max_leaf_nodes': 195, 'n_estimators': 499}. Best is trial 29 with value: 1.7227708338086274.
[I 2022-10-28 16:47:02,998] Trial 41 finished with value: 1.7225648611517552 and parameters: {'max_depth': 10, 'max_leaf_nodes': 547, 'n_estimators': 189}. Best is trial 41 with value: 1.7225648611517552.
[I 2022-10-28 16:47:51,405] Trial 42 finished with value: 1.7331377702462532 and parameters: {'max_depth': 10, 'max_leaf_nodes': 607, 'n_estimators': 163}. Best is trial 41 with value: 

[I 2022-10-28 17:36:13,615] Trial 76 finished with value: 1.7330004931882173 and parameters: {'max_depth': 10, 'max_leaf_nodes': 471, 'n_estimators': 170}. Best is trial 53 with value: 1.7141577632583964.
[I 2022-10-28 17:36:47,328] Trial 77 finished with value: 1.742183928882649 and parameters: {'max_depth': 9, 'max_leaf_nodes': 901, 'n_estimators': 124}. Best is trial 53 with value: 1.7141577632583964.
[I 2022-10-28 17:37:00,177] Trial 78 finished with value: 2.3241937885704913 and parameters: {'max_depth': 2, 'max_leaf_nodes': 582, 'n_estimators': 135}. Best is trial 53 with value: 1.7141577632583964.
[I 2022-10-28 17:38:44,664] Trial 79 finished with value: 1.7252688750893594 and parameters: {'max_depth': 10, 'max_leaf_nodes': 735, 'n_estimators': 355}. Best is trial 53 with value: 1.7141577632583964.
[I 2022-10-28 17:41:04,454] Trial 80 finished with value: 1.7221863130379382 and parameters: {'max_depth': 10, 'max_leaf_nodes': 694, 'n_estimators': 471}. Best is trial 53 with value

Best Score: 1.712158913246908
Best trial: {'max_depth': 10, 'max_leaf_nodes': 803, 'n_estimators': 439}


In [15]:
kf = KFold(n_splits = 5)

cnt = 1
nrmse = 0
total_nrmse = 0

for train_idx, valid_idx in kf.split(train_x):
    train_data = train.iloc[train_idx]
    valid_data = train.iloc[valid_idx]
    
    rf = RandomForestRegressor(**study.best_trial.params)
    
    model = MultiOutputRegressor(estimator=rf)
    
    x_train = train_data.iloc[:,0:18]
    y_train = train_data.iloc[:,18:24]
    model.fit(x_train,y_train)
        
    x_valid = valid_data.iloc[:,0:18]
    y_valid = valid_data.iloc[:,18:24]
    preds = model.predict(x_valid)
    
    fit_params = dict(eval_set = [(x_train, y_train)], eval_metric = 'rmse')
    
    nrmse = nrmse_1(preds, y_valid)
    
    print(cnt," 번째 모델 nrmse : ",nrmse)
    
    total_nrmse += nrmse
    cnt+=1



print('모델 nrmse 평균 : ',total_nrmse/5)

1  번째 모델 nrmse :  1.4546006556507636
2  번째 모델 nrmse :  2.2087128097451183
3  번째 모델 nrmse :  1.5807766113130532
4  번째 모델 nrmse :  1.47284942168641
5  번째 모델 nrmse :  1.893931465227577
모델 nrmse 평균 :  1.7221741927245844


In [16]:
rf_guro = RandomForestRegressor(**study.best_trial.params)
model_guro_rf = MultiOutputRegressor(estimator=rf_guro)

In [17]:
model_guro_rf

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=10,
                                                     max_leaf_nodes=803,
                                                     n_estimators=439))

In [18]:
model_guro_rf.fit(train.iloc[train_idx].iloc[:,0:18], train.iloc[train_idx].iloc[:,18:24])

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=10,
                                                     max_leaf_nodes=803,
                                                     n_estimators=439))

In [21]:
guro_x

,1월가스,1월온실가스,2월가스,2월온실가스,3월가스,3월온실가스,4월가스,4월온실가스,5월가스,5월온실가스,6월가스,6월온실가스,7월가스,7월온실가스,8월가스,8월온실가스,9월가스,9월온실가스
0,2.979134e+06,6489.435778,2.660926e+06,5796.284195,1.801233e+06,3923.618241,1.141260e+06,2486.003023,763480.306265,1663.086303,525004.229698,1143.614755,389923.689095,849.369318,311298.445476,678.100243,331510.257541,722.127557
1,1.381606e+06,3009.547055,1.209618e+06,2634.907409,8.678105e+05,1890.348388,5.352239e+05,1165.876165,351540.647332,765.759681,257359.373550,560.604963,192825.846868,420.031823,179970.232019,392.028485,162546.178654,354.073735
2,2.683260e+06,5844.935497,2.334458e+06,5085.142124,1.761741e+06,3837.593424,1.084778e+06,2362.968417,826332.628770,1799.997283,620818.482599,1352.326585,570627.255220,1242.995222,513581.129930,1118.731860,298433.663573,650.076936
3,2.611342e+06,5688.275725,2.162210e+06,4709.933073,1.675993e+06,3650.808896,1.188879e+06,2589.729918,939166.111369,2045.782037,758622.679814,1652.504954,741244.053364,1614.649157,745125.067285,1623.103155,683740.092807,1489.388494
4,1.782925e+06,3883.738614,1.521106e+06,3313.418783,1.125252e+06,2451.133104,7.297497e+05,1589.610955,510736.361949,1112.535112,331206.995360,721.466963,231258.918794,503.750440,209618.823666,456.611902,93426.515081,203.510629
5,3.077955e+06,6704.697860,2.580908e+06,5621.982254,1.691166e+06,3683.861343,1.158290e+06,2523.099540,736413.076566,1604.125858,561356.032483,1222.799752,474897.401392,1034.467238,391710.856148,853.262297,377477.858469,822.258611
6,1.093541e+06,2382.057151,9.196714e+05,2003.316748,6.868636e+05,1496.192475,4.660213e+05,1015.132534,317387.881671,691.364839,205840.512761,448.381621,142964.009281,311.417968,114366.501160,249.124123,274676.791183,598.327430
7,1.663826e+06,3624.306591,1.414538e+06,3081.282202,9.954745e+05,2168.438443,6.883796e+05,1499.494752,457040.350348,995.569290,351553.276102,765.787190,306243.064965,667.088126,248666.672854,541.669686,190195.299304,414.301711
8,1.639530e+06,3571.383024,1.370625e+06,2985.626284,9.496408e+05,2068.599079,6.198521e+05,1350.221578,415702.888631,905.524052,276462.587007,602.217422,190443.201856,414.841716,161028.464037,350.767703,114863.721578,250.207216
9,2.064783e+06,4497.708228,1.775909e+06,3868.455440,1.265381e+06,2756.374414,8.279172e+05,1803.448867,534516.904872,1164.336180,342378.807425,745.802479,243259.143852,529.890486,212262.517401,462.370650,211420.255220,460.535953


In [20]:
preds_rf = model_guro_rf.predict(guro_x) ### 값 변경 부분

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
guro_y.iloc[:,0] = preds_rf[:,0]
guro_y.iloc[:,1] = preds_rf[:,1]
guro_y.iloc[:,2] = preds_rf[:,2]
guro_y.iloc[:,3] = preds_rf[:,3]
guro_y.iloc[:,4] = preds_rf[:,4]
guro_y.iloc[:,5] = preds_rf[:,5]

In [ ]:
guro_y

In [ ]:
guro_2020 = pd.concat([guro_x, guro_y], axis=1)

guro_2020

In [ ]:
guro_2020.to_csv('guro_2020_rf.csv', index=False, encoding="cp949")